In [1]:
import pandas as pd

## Steps

- Read population of each county in the USA

- Load production per category per county in the USA

- Load average needs: Vegetables + Grains + Animal Protein. Scale by population.

- Subtract production and needs. There will be surplus and deficit.

- Find a way to match surplus and deficit, and to penalize for the distance.

## Assumptions

- Yield per acre is uniform accross farms.

## Data Exploration

In [3]:
local = pd.read_excel('data/DataDownload.xls',sheetname='LOCAL')

In [4]:
local.head()

,FIPS,State,County,DIRSALES_FARMS07,PCT_LOCLFARM07,PCT_LOCLSALE07,DIRSALES07,PC_DIRSALES07,FMRKT09,FMRKT13,...,BERRY_ACRESPTH07,SLHOUSE07,GHVEG_FARMS07,GHVEG_SQFT07,GHVEG_SQFTPTH07,FOODHUB12,CSA07,AGRITRSM_OPS07,AGRITRSM_RCT07,FARM_TO_SCHOOL
0,1001,AL,Autauga,25.0,6.0,0.6,100.0,2.01,2.0,1.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2.0,7.0,228000.0,0.0
1,1003,AL,Baldwin,80.0,7.0,0.7,715.0,4.16,4.0,4.0,...,0.457136,1.0,0.0,0.0,0.0,0.0,13.0,18.0,124000.0,0.0
2,1005,AL,Barbour,18.0,2.9,0.0,11.0,0.37,2.0,3.0,...,NaN,0.0,0.0,0.0,0.0,0.0,1.0,27.0,163000.0,0.0
3,1007,AL,Bibb,12.0,5.7,NaN,46.0,2.14,1.0,1.0,...,0.279265,0.0,0.0,0.0,0.0,0.0,2.0,5.0,NaN,0.0
4,1009,AL,Blount,84.0,5.9,0.3,429.0,7.59,1.0,1.0,...,0.492386,0.0,2.0,NaN,NaN,0.0,7.0,10.0,293000.0,0.0


In [5]:
local.columns

Index([u'FIPS', u'State', u'County', u'DIRSALES_FARMS07', u'PCT_LOCLFARM07',
       u'PCT_LOCLSALE07', u'DIRSALES07', u'PC_DIRSALES07', u'FMRKT09',
       u'FMRKT13', u'PCH_FMRKT_09_13', u'FMRKTPTH09', u'FMRKTPTH13',
       u'PCH_FMRKTPTH_09_13', u'FMRKT_SNAP13', u'PCT_FMRKT_SNAP13',
       u'FMRKT_WIC13', u'PCT_FMRKT_WIC13', u'FMRKT_WICCASH13',
       u'PCT_FMRKT_WICCASH13', u'FMRKT_SFMNP13', u'PCT_FMRKT_SFMNP13',
       u'FMRKT_FRVEG13', u'PCT_FRMKT_FRVEG13', u'FMRKT_ANMLPROD13',
       u'PCT_FRMKT_ANMLPROD13', u'FMRKT_OTHER13', u'PCT_FMRKT_OTHER13',
       u'VEG_FARMS07', u'VEG_ACRES07', u'VEG_ACRESPTH07', u'FRESHVEG_FARMS07',
       u'FRESHVEG_ACRES07', u'FRESHVEG_ACRESPTH07', u'ORCHARD_FARMS07',
       u'ORCHARD_ACRES07', u'ORCHARD_ACRESPTH07', u'BERRY_FARMS07',
       u'BERRY_ACRES07', u'BERRY_ACRESPTH07', u'SLHOUSE07', u'GHVEG_FARMS07',
       u'GHVEG_SQFT07', u'GHVEG_SQFTPTH07', u'FOODHUB12', u'CSA07',
       u'AGRITRSM_OPS07', u'AGRITRSM_RCT07', u'FARM_TO_SCHOOL'],
      dtype

In [17]:
vegetables = local[['FIPS','State', 'County','VEG_ACRES07', 'FRESHVEG_ACRES07', 'GHVEG_FARMS07']]

In [20]:
len(pd.unique(vegetables['County']))

1941

In [18]:
non_na_vegetables = vegetables.dropna()

In [13]:
non_na_vegetables['VEG_ACRES07'] >= non_na_vegetables['FRESHVEG_ACRES07']

2       True
4       True
7       True
8       True
10      True
11      True
12      True
14      True
15      True
17      True
18      True
19      True
22      True
23      True
27      True
32      True
33      True
36      True
41      True
42      True
44      True
46      True
47      True
48      True
51      True
52      True
55      True
58      True
59      True
60      True
        ... 
3134    True
3135    True
3136    True
3137    True
3138    True
3139    True
3140    True
3141    True
3142    True
3143    True
3146    True
3147    True
3148    True
3149    True
3150    True
3151    True
3152    True
3155    True
3157    True
3160    True
3163    True
3165    True
3168    True
3170    True
3171    True
3172    True
3173    True
3174    True
3175    True
3176    True
dtype: bool

In [14]:
non_na_vegetables['VEG_ACRES07'].sum()

3740457.0

In [16]:
non_na_vegetables['FRESHVEG_ACRES07'].

(1796,)